In [1]:
# torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import transforms
import torch.optim as optim
# utils
import os
from glob import glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

In [3]:
class MyModel(nn.Module):
    def __init__(self, num_classes=2):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6*6*128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "C:\\Users\\sangeetha\\bone_10"  # Change this to your actual model path
# Set the model to evaluation mode
num_classes=2
model = MyModel(num_classes).to(device)
model.load_state_dict(torch.load("C:\\Users\\sangeetha\\bone_10"))  # Replace with the correct model file name
model.eval()  # Set the model to evaluation mode


C:\Users\sangeetha\AppData\Local\Temp\ipykernel_4200\3310834324.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("C:\\Users\\sangeetha\\b

MyModel(
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, in

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Open and convert to RGB
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension
    return image.to(device)


In [6]:
label_dict = {
    
    0:"fractured",
    1: "not fractured",
  
  
}

def predict(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class index
    return label_dict[predicted.item()]  # Return the class name

In [7]:
image_path = "C:\\Users\\sangeetha\\1\\Bone_Fracture_Binary_Classification\\dataset\\Bone_Fracture_Binary_Classification\\test\\fractured\\2-rotated2-rotated3 - Copy.jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: fractured


In [8]:
image_path = "C:\\Users\\sangeetha\\1\\Bone_Fracture_Binary_Classification\\dataset\\Bone_Fracture_Binary_Classification\\test\\not fractured\\2.png"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: not fractured


In [9]:

image_path = "C:\\Users\\sangeetha\\1\\Bone_Fracture_Binary_Classification\\dataset\\Bone_Fracture_Binary_Classification\\val\\fractured\\2-rotated1-rotated2-rotated3.jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")


Predicted  Type: fractured


In [10]:


image_path = "C:\\Users\\sangeetha\\1\\Bone_Fracture_Binary_Classification\\dataset\\Bone_Fracture_Binary_Classification\\val\\not fractured\\1-rotated3.jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: not fractured


In [11]:
image_path = "Downloads\\OIP (4).jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: fractured


In [12]:
image_path = "Downloads\\OIP (5).jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: fractured


In [13]:
image_path = "Downloads\\4d83d807fe7af170aa25b936567aa7e4.jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: fractured


In [14]:
image_path = "Downloads\\OIP (6).jpg"  # Change this to your test image path
prediction = predict(image_path)
print(f"Predicted  Type: {prediction}")

Predicted  Type: fractured
